# Proyek Machine Learning
M. Ardifa Rizqi Ramadhan

## Import Library

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

## Import Dataset

In [ ]:
url = 'https://raw.githubusercontent.com/ardifa24/Machine-Learning-Terapan/refs/heads/main/Proyek-Pertama/CarPricesPrediction.csv'
carPrices = pd.read_csv(url)
carPrices.head()

## Data Understanding

In [ ]:
# Menghapus Kolom Unnamed: 0, karena hanya index dari file CSV
carPrices = carPrices.drop('Unnamed: 0', axis=1)

In [ ]:
carPrices.head()

In [ ]:
carPrices.info()

Berdasarkan informasi diatas menunjukan bahwa:
*  Tipe data object menunjukkan bahwa kolom tersebut berisi data kategorikal dan perlu dikodekan (misalnya dengan One Hot Encoding) sebelum digunakan dalam pemodelan machine learning.
*  Tidak terdapat data yang hilang (Non-Null Count = 1000 untuk semua kolom).

*  Dataset ini cocok untuk kasus regresi, karena target (Price) bersifat kontinu.

In [ ]:
carPrices.describe()

Berdasarkan informasi diatas menunjukkan bahwa:
*  Tahun mobil berkisar antara 2010 hingga 2022, dengan median di 2016. Ini menunjukkan mayoritas mobil relatif baru.
*  Kisaran jarak tempuh sangat bervariasi (10 ribu hingga hampir 150 ribu km), menunjukkan dataset mencakup mobil baru hingga cukup tua.
*  Harga mobil tersebar antara sekitar 12 ribu hingga 31 ribu, dengan rata-rata sekitar 22 ribu. Ini cocok untuk kasus regresi harga mobil.

In [ ]:
carPrices.isnull().sum()

Tidak terdapat missing values pada keenam kolom (Make, Model, Year, Mileage, Condition, dan Price) dalam dataset.

In [ ]:
carPrices.duplicated().sum()

Tidak terdapat baris data yang duplikat di dalam dataset carPrice

Mengecek nilai Outlier

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

sns.boxplot(x=carPrices['Mileage'], ax=axes[0])
axes[0].set_title('Mileage')

sns.boxplot(x=carPrices['Year'], ax=axes[1])
axes[1].set_title('Year')

sns.boxplot(x=carPrices['Price'], ax=axes[2])
axes[2].set_title('Price')

plt.tight_layout()
plt.show()

Berdasarkan visualisasi diatas menunjukkan bahwa ketiga fitur numerik (Mileage, Year,
 Price) memiliki distribusi yang relatif normal/simetris

In [ ]:
numerical_features = ['Year','Mileage','Price']
categorical_features = ['Make','Model','Condition']

In [ ]:
feature = categorical_features[0]
count = carPrices[feature].value_counts()
percent = 100*carPrices[feature].value_counts(normalize=True)
df = pd.DataFrame({'jumlah sampel':count, 'persentase':percent.round(1)})
print(df)
count.plot(kind='bar', title=feature);

Berdasarkan Visualisasi diatas menunjukkan bahwa:
*  Chevrolet adalah merek paling umum dalam dataset, disusul oleh Toyota.
*  Ford dan Honda memiliki jumlah yang sama (199 sampel).
*  Nissan merupakan merek dengan frekuensi paling rendah, namun perbedaannya sangat tipis.
*  Distribusi merek cukup merata, masing-masing menyumbang sekitar 18–21% dari total data.

In [ ]:
feature = categorical_features[1]
count = carPrices[feature].value_counts()
percent = 100*carPrices[feature].value_counts(normalize=True)
df = pd.DataFrame({'jumlah sampel':count, 'persentase':percent.round(1)})
print(df)
count.plot(kind='bar', title=feature);

Berdasarkan visualisasi diatas menunjukkan bahwa:
*  Altima adalah model dengan jumlah sampel terbanyak (226 unit), berkontribusi sekitar 22.6% dari total data.
*  Model lain seperti Camry dan Silverado juga memiliki representasi signifikan di dataset.
*  Distribusi model cukup seimbang sehingga cocok untuk dijadikan fitur dalam pemodelan prediktif, misalnya regresi harga mobil.



In [ ]:
feature = categorical_features[2]
count = carPrices[feature].value_counts()
percent = 100*carPrices[feature].value_counts(normalize=True)
df = pd.DataFrame({'jumlah sampel':count, 'persentase':percent.round(1)})
print(df)
count.plot(kind='bar', title=feature);

Berdasarkan visualisasi diatas menunjukkan bahwa:
*  Mayoritas mobil (59.5%) berada dalam kondisi Excellent, menunjukkan bahwa sebagian besar kendaraan dalam dataset ini berada dalam kondisi sangat baik.
*  Hanya 11.3% mobil yang berada dalam kondisi Fair

In [ ]:
mean_prices = carPrices.groupby('Make')['Price'].mean().sort_values(ascending=False)

plt.figure(figsize=(8,6))
bars = mean_prices.plot(kind='bar', color='steelblue')
plt.title('Average Car Price by Make')
plt.ylabel('Price')
plt.xlabel('Make')

for i, v in enumerate(mean_prices):
    plt.text(i, v + 200, f'{v:.0f}', ha='center', fontsize=9)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
carPrices.hist(bins=50, figsize=(13,8))
plt.show()


*   Pada Fitur Year yang merupakan tahun produksi mobil, menunjukkan distribusi terlihat cukup merata pada tahun 2010 hingga 2022, dengan puncak jumlah kendaraan berapa di tahun 2011 dan 2015.
*   Pada fitur Mileage yang merupakan jarak tempuh mobil, menunjukan bahwa distribusi terlihat cukup menyebar, mulai dari sekitar 10.000 km hingga lebih dari 140.000 km
*   Pada fitur Prices yang merupakan harga mobil, menunjukkan bahwa sebagian besar mobil berada pada kisaran harga pasar menengah


Mengecek Fitur apa saja yang memiliki korelasi dengan price

In [ ]:
sns.pairplot(carPrices, diag_kind = 'kde')

In [ ]:
correlation_matrix = carPrices[numerical_features].corr()

plt.figure(figsize=(6, 4))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Heatmap Korelasi Fitur Numerik dengan Price")
plt.tight_layout()
plt.show()

Berdasarkan visualisasi heatmap korelasi diatas menunjukkan bahwa:
*  Fitur Year memiliki pengaruh paling besar terhadap Price, dengan korelasi negatif yang sangat kuat.
* Mileage juga berpengaruh terhadap Price, meskipun tidak sekuat Year.


## Data Preparation

*   **One Hot Encoding** untuk Mengubah fitur kategorikal (berupa teks) menjadi fitur baru yang sesuai sehingga dapat mewakili variabel kategori
*   **Feature target Split** untuk Memisahkan data menjadi dua bagian: fitur (X) dan target prediksi (y).
*   **Split data 80% data train dan 20% data Test** untuk Membagi data menjadi dua subset: data pelatihan (training) dan data pengujian (testing), agar bisa mengukur performa model secara objektif

In [ ]:
#One Hot Encoding
carPrices = pd.concat([carPrices, pd.get_dummies(carPrices['Make'], prefix='Make')], axis=1)
carPrices = pd.concat([carPrices, pd.get_dummies(carPrices['Model'], prefix='Model')], axis=1)
carPrices = pd.concat([carPrices, pd.get_dummies(carPrices['Condition'], prefix='Condition')], axis=1)
carPrices.drop(['Make', 'Model', 'Condition'], axis=1, inplace=True)

carPrices.head()

In [ ]:
X = carPrices.drop(columns='Price')
y = carPrices['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Model Development

*   Linear Regression
*   Random Forest
*   SVR


### Linier Regression

In [ ]:
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

y_pred_lr = lr_model.predict(X_test)

r2_lr = r2_score(y_test, y_pred_lr)
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))
mae_lr = mean_absolute_error(y_test, y_pred_lr)

print(f"Linear Regression -> R²: {r2_lr:.3f}, RMSE: {rmse_lr:.2f}, MAE: {mae_lr:.2f}")

### Random Forest

In [ ]:
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)

r2_rf = r2_score(y_test, y_pred_rf)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
mae_rf = mean_absolute_error(y_test, y_pred_rf)

print(f"Random Forest -> R²: {r2_rf:.3f}, RMSE: {rmse_rf:.2f}, MAE: {mae_rf:.2f}")

### Support Vector Regressor (SVR)

In [ ]:
svr_model = SVR()
svr_model.fit(X_train, y_train)

y_pred_svr = svr_model.predict(X_test)

r2_svr = r2_score(y_test, y_pred_svr)
rmse_svr = np.sqrt(mean_squared_error(y_test, y_pred_svr))
mae_svr = mean_absolute_error(y_test, y_pred_svr)

print(f"SVR -> R²: {r2_svr:.3f}, RMSE: {rmse_svr:.2f}, MAE: {mae_svr:.2f}")

In [ ]:
evaluasi_model = {
    'Model': ['Linear Regression', 'Random Forest', 'SVR'],
    'R² Score': [r2_lr, r2_rf, r2_svr],
    'RMSE': [rmse_lr, rmse_rf, rmse_svr],
    'MAE': [mae_lr, mae_rf, mae_svr]
}

df_evaluasi = pd.DataFrame(evaluasi_model)

print(df_evaluasi)

Berdasarkan evaluasi menggunakan metrik R2, RMSE, dan MAE, model Linear Regression memiliki performa terbaik dalam memprediksi harga mobil

In [ ]:
models = {
    'Linear Regression': y_pred_lr,
    'Random Forest': y_pred_rf,
    'SVR': y_pred_svr
}

plt.figure(figsize=(18, 5))

for i, (name, y_pred) in enumerate(models.items()):
    plt.subplot(1, 3, i+1)
    plt.scatter(y_test, y_pred, alpha=0.6, edgecolor='k')
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    plt.title(name)
    plt.xlabel("Actual Price")
    plt.ylabel("Predicted Price")
    plt.grid(True)

plt.tight_layout()
plt.show()

Berdasarkan visualisasi dan evaluasi:

*   Linear Regression merupakan model dengan performa terbaik dan paling stabil.
*   Random Forest menunjukkan hasil yang cukup baik, meskipun sedikit lebih menyebar.
*   SVR memerlukan penyesuaian lanjutan sebelum layak digunakan.

### Testing

In [ ]:
sample = X_test.iloc[[1]]
actual_price = y_test.iloc[1]

pred_lr = lr_model.predict(sample)[0]
pred_rf = rf_model.predict(sample)[0]
pred_svr = svr_model.predict(sample)[0]

print("=== Prediksi Harga Mobil ===")
print(f"Harga Aktual        : ${actual_price:.2f}")
print(f"Linear Regression   : ${pred_lr:.2f}")
print(f"Random Forest       : ${pred_rf:.2f}")
print(f"Support Vector Reg. : ${pred_svr:.2f}")

Dilakukan Pengujian lebih lanjut untuk memastikan bahwa model bekerja dengan baik dan menunjukkan bahwa prediksi Harga mobil dengan Linier Regression memberikan estimasi yang sangat mendekati nilai aktual, dengan selisih hanya sebesar $0.03